# Analysing human rights classifications

#### Imports

In [2]:
# !pip install openpyxl

In [3]:
import pandas as pd
import openpyxl
import glob

#### Functions

In [4]:
########################################################
# AI generated, but checked and modified
########################################################

from openpyxl.styles import Color

def _get_color_identifier(color_obj):
    """
    Internal helper to get a reliable identifier for any openpyxl color object.
    Handles standard RGB, theme, and indexed colors.
    Returns a tuple like ('rgb', 'FFFF0000') or ('theme', 5).
    """
    if not isinstance(color_obj, Color):
        return None
    
    try:
        if color_obj.rgb:
            return ('rgb', color_obj.rgb)
    except TypeError:
        pass  # This happens with theme colors, so we fall back

    if color_obj.type == 'theme':
        return ('theme', color_obj.theme)
    if color_obj.type == 'indexed':
        return ('indexed', color_obj.indexed)
    return None

def get_color_identifier(color_obj):
    """
    Tries to get a unique identifier for a color, preferring RGB,
    but falling back to theme or indexed color if RGB fails.
    (This function does not need to be changed)
    """
    if not isinstance(color_obj, Color):
        return None
    try:
        if color_obj.rgb:
            return ('rgb', color_obj.rgb)
    except TypeError:
        pass  # This happens with theme colors, so we fall back
    if color_obj.type == 'theme':
        return ('theme', color_obj.theme)
    if color_obj.type == 'indexed':
        return ('indexed', color_obj.indexed)
    return None

def find_font_colors(file_path: str, sheet: str = 'Sheet1', col_name: str = 'human_rights') -> None:
    """Inspects a column and reports detailed FILL/BACKGROUND color information."""
    try:
        workbook = openpyxl.load_workbook(file_path, data_only=True)
        worksheet = workbook[sheet]
    except Exception as e:
        print(f"Error loading workbook: {e}")
        return

    header = [cell.value for cell in worksheet[1]]
    try:
        col_idx = header.index(col_name) + 1
    except ValueError:
        print(f"Error: Column '{col_name}' not found in the header row.")
        return

    print(f"--- Discovering Fill/Background Colors in Column: '{col_name}' ---\n")
    
    found_colors = {}
    for row in range(2, worksheet.max_row + 1):
        cell = worksheet.cell(row=row, column=col_idx)
        
        # *** THIS IS THE ONLY MAJOR CHANGE ***
        # We now look at the cell's fill color instead of font color
        if cell.fill and cell.fill.fgColor:
            identifier = get_color_identifier(cell.fill.fgColor)
            if identifier:
                cell_value = cell.value
                if identifier not in found_colors:
                    found_colors[identifier] = set()
                if len(found_colors[identifier]) < 5: # Get a few examples
                     found_colors[identifier].add(cell_value)

    if not found_colors:
        print("No unique fill/background colors found in this column.")
    else:
        print("Found the following color identifiers. Use these in your main script's map:")
        for identifier, values in found_colors.items():
            id_type, id_value = identifier
            print(f"  - Identifier Type: '{id_type}', Value: {id_value}")
            print(f"    (Used for cells with text like: {list(values)})\n")

def categorize_by_color(
    excel_file_path: str,
    column_to_check: str,
    category_map: dict,
    color_type: str = 'fill',
    sheet_name: str | int = 0,
    new_category_column: str = 'Category',
    default_category: str = 'Uncategorized'
) -> pd.DataFrame:
    """
    Categorizes rows in an Excel file based on cell color and adds a new category column.

    Args:
        excel_file_path (str): Path to the Excel file.
        column_to_check (str): The name of the column with colored cells.
        category_map (dict): A dictionary mapping color identifiers to category names.
                             Example: {('theme', 5): 'Core Right', ('rgb', 'FFFF0000'): 'Urgent'}
        color_type (str, optional): The type of color to check. Either 'fill' for background
                                    or 'font' for text color. Defaults to 'fill'.
        sheet_name (str | int, optional): The name or index of the sheet to read. Defaults to 0 (the first sheet).
        new_category_column (str, optional): The name for the new category column. Defaults to 'Category'.
        default_category (str, optional): The category for uncolored or unmapped cells. Defaults to 'Uncategorized'.

    Returns:
        pd.DataFrame: A new pandas DataFrame with the added category column.
    
    Raises:
        FileNotFoundError: If the excel_file_path is invalid.
        KeyError: If the sheet_name is not found.
        ValueError: If the column_to_check is not found or color_type is invalid.
    """
    if color_type not in ['fill', 'font']:
        raise ValueError("color_type must be either 'fill' or 'font'.")

    # 1. Load data and workbook
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    workbook = openpyxl.load_workbook(excel_file_path)
    sheet = workbook[sheet_name] if isinstance(sheet_name, str) else workbook.worksheets[sheet_name]

        # 2. Find column index
    header = [cell.value for cell in sheet[1]]
    if column_to_check not in header:
        raise ValueError(f"Column '{column_to_check}' not found in the Excel file header.")
    col_idx = header.index(column_to_check) + 1

    # 3. Iterate and build the category list
    categories = []
    for row_index in range(2, sheet.max_row + 1):
        cell = sheet.cell(row=row_index, column=col_idx)
        
        category = default_category
        color_obj = None
        
        if color_type == 'fill':
            if cell.fill and cell.fill.fgColor:
                color_obj = cell.fill.fgColor
        elif color_type == 'font':
            if cell.font and cell.font.color:
                color_obj = cell.font.color

        if color_obj:
            identifier = _get_color_identifier(color_obj)
            if identifier in category_map:
                category = category_map[identifier]
                
        categories.append(category)

    # 4. Add the new column to the DataFrame
    if len(categories) != len(df):
        print(f"Warning: Row count mismatch. Pandas read {len(df)} rows, openpyxl found {len(categories)}. Truncating/padding to match pandas.")
    df[new_category_column] = pd.Series(categories, index=df.index)

    return df

#### Main

In [5]:
find_font_colors('annotated_rights/scanbike_(Qwen3)_human_rights.xlsx')

--- Discovering Fill/Background Colors in Column: 'human_rights' ---

Found the following color identifiers. Use these in your main script's map:
  - Identifier Type: 'rgb', Value: FFFF0000
    (Used for cells with text like: ['Article 1 (Right to Property)', 'Article 2 (Right to Life)', 'Article 1 (Duty to Respect Human Rights)', 'Article 17 (Right to Property)', 'Article 2 (Right to life)'])

  - Identifier Type: 'rgb', Value: 00000000
    (Used for cells with text like: ['Article 6 (Right to a fair trial)', 'Article 13 (Effective Remedy)', 'Article 1 (Right to peaceful enjoyment of possessions)', 'Article 6 (Right to a Fair Trial)', 'Article 6 (Right to Fair Trial)'])

  - Identifier Type: 'rgb', Value: FFFFFF00
    (Used for cells with text like: ['Article 1 of Protocol 1 (Right to Peaceful Enjoyment of Possessions)'])



In [6]:
# Step 1: Define your file and column settings
FILE_PATH = 'annotated_rights/IT_(Gemma3)_human_rights.xlsx'
SHEET = 'Sheet1'  # Or use 0 for the first sheet
COLUMN = 'human_rights'
NEW_COLUMN_NAME = 'Rights_Category'

# Step 2: Define your mapping of colors to categories.
# You get these identifiers by running the discovery script first.
# This example assumes we are checking BACKGROUND/FILL color.
FILL_COLOR_MAP = {
    ('rgb', '00000000'): 'Meaningful', 
    ('rgb', 'FFFFFF00'): 'Meaningful (wrong article number)', # Yellow
    ('rgb', 'FF92D050'): 'Non meaningful (does not apply to stakeholder)',  # Green
    ('rgb', 'FFFF0000'): 'Non meaningful' # Red
}

# Step 3: Call the function with your settings
try:
    print(f"Categorizing based on FILL color in '{COLUMN}'...")
    categorized_df = categorize_by_color(
        excel_file_path=FILE_PATH,
        column_to_check=COLUMN,
        category_map=FILL_COLOR_MAP,
        color_type='fill',  # Explicitly state we want fill color
        sheet_name=SHEET,
        new_category_column=NEW_COLUMN_NAME
    )

    # Step 4: Display the results
    print("\n--- Categorization Complete! ---")
    print("First 5 rows of the new DataFrame:")
    display(categorized_df.head())
    
    print(f"\nValue counts for the new '{NEW_COLUMN_NAME}' column:")
    print(categorized_df[NEW_COLUMN_NAME].value_counts())
    
    # Optionally, save the output
    # categorized_df.to_csv('categorized_output.csv', index=False)
    # print("\nSaved categorized data to 'categorized_output.csv'")

except (FileNotFoundError, KeyError, ValueError) as e:
    print(f"\nAn error occurred: {e}")

Categorizing based on FILL color in 'human_rights'...

--- Categorization Complete! ---
First 5 rows of the new DataFrame:


,stakeholder,problematic_behaviour,harm,human_rights,is_human_right_affected,Rights_Category
0,the applicant,False Positives,Imagine you are the applicant; you may be unfa...,Article 14 (Prohibition of Discrimination),NaN,Meaningful
1,the applicant,False Positives,Imagine you are the applicant; you may be unfa...,Article 8 (Right to Private and Family Life),NaN,Meaningful
2,the applicant,False Positives,Imagine you are the applicant; you may be unfa...,Article 1 (Protection of Property) PROTOCOL!!,NaN,Meaningful (wrong article number)
3,the applicant,False Positives,Imagine you are the applicant; you may be unfa...,Article 6 (Right to a Fair Trial),NaN,Meaningful
4,the applicant,False Negatives,Imagine you are the applicant; despite being f...,Article 1 (Protection of Property and Peaceful...,NaN,Meaningful



Value counts for the new 'Rights_Category' column:
Rights_Category
Meaningful                                        428
Non meaningful                                     22
Meaningful (wrong article number)                   3
Non meaningful (does not apply to stakeholder)      1
Name: count, dtype: int64


## Definitive code

In [7]:
# Step 1: Define your file and column settings
SHEET = 'Sheet1'  # Or use 0 for the first sheet
COLUMN = 'human_rights'
NEW_COLUMN_NAME = 'Rights_Category'

# Step 2: Define your mapping of colors to categories.
# You get these identifiers by running the discovery script first.
# This example assumes we are checking BACKGROUND/FILL color.
FILL_COLOR_MAP = {
    ('rgb', '00000000'): 'Meaningful', 
    ('rgb', 'FFFFFF00'): 'Meaningful (wrong article number)', # Yellow
    ('rgb', 'FF92D050'): 'Non meaningful (does not apply to stakeholder)',  # Green
    ('rgb', 'FFFF0000'): 'Non meaningful' # Red
}

total_rights = 0

for FILE_PATH in glob.glob('annotated_rights/*human_rights.xlsx'):
    # Step 3: Call the function with your settings
    print(f"\nProcessing file: {FILE_PATH}")

    try:
        # print(f"Categorizing based on FILL color in '{COLUMN}'...")
        categorized_df = categorize_by_color(
            excel_file_path=FILE_PATH,
            column_to_check=COLUMN,
            category_map=FILL_COLOR_MAP,
            color_type='fill',  # Explicitly state we want fill color
            sheet_name=SHEET,
            new_category_column=NEW_COLUMN_NAME
        )

        # Step 4: Display the results
        # print("\n--- Categorization Complete! ---")
        # print("First 5 rows of the new DataFrame:")
        # display(categorized_df.head())
        
        # print(f"\nValue counts for the new '{NEW_COLUMN_NAME}' column:")
        value_counts = categorized_df[NEW_COLUMN_NAME].value_counts()
        # print(value_counts)
        
        n_meaningful = value_counts.get('Meaningful', 0) + value_counts.get('Meaningful (wrong article number)', 0)
        n_non_meaningful = value_counts.get('Non meaningful', 0) + value_counts.get('Non meaningful (does not apply to stakeholder)', 0)
        total_rights += len(categorized_df)

        # print(f"\nTotal Meaningful Rights: {n_meaningful}")
        # print(f"Total Non-Meaningful Rights: {n_non_meaningful}")
        print(f"Total Rights: {len(categorized_df)}")

        print(f"Percentage of Meaningful Rights: {n_meaningful / len(categorized_df) * 100:.2f}%\n\n")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"\nAn error occurred: {e}")

print(f"\nTotal Rights Processed Across All Files: {total_rights}")


Processing file: annotated_rights/IT_(Qwen3)_human_rights.xlsx
Total Rights: 298
Percentage of Meaningful Rights: 95.97%



Processing file: annotated_rights/IT_RAG_(Mistral_Small)_human_rights.xlsx
Total Rights: 338
Percentage of Meaningful Rights: 91.72%



Processing file: annotated_rights/scanbike_RAG_(Gemma3)_human_rights.xlsx
Total Rights: 402
Percentage of Meaningful Rights: 95.52%



Processing file: annotated_rights/scanbike_RAG_(Qwen3)_human_rights.xlsx
Total Rights: 274
Percentage of Meaningful Rights: 98.91%



Processing file: annotated_rights/scanbike_(Qwen3)_human_rights.xlsx
Total Rights: 338
Percentage of Meaningful Rights: 97.93%



Processing file: annotated_rights/scanbike_(Mistral_Small)_human_rights.xlsx
Total Rights: 504
Percentage of Meaningful Rights: 92.66%



Processing file: annotated_rights/scanbike_RAG_(Mistral_Small)_human_rights.xlsx
Total Rights: 378
Percentage of Meaningful Rights: 91.01%



Processing file: annotated_rights/IT_(Gemma3)_human_rights.x